#### Parameters:
$a_i =$ the total return from asset i

$sec =$ the indices of possible investments.

$time =$ a list of the investment years.

$initfund = $ the size of the original portfolio.

#### Decision Variables:

$x_{ij} $ the amount of money invested in option i in year j.

$fund_j =$ The amount available to be invested at the beginning of year $j=1,\dots, 8$.


#### Objective Function:

Maximize the size of the portfolio in the beginning of year 8.
Max: $fund_8$


#### Constraints:

nonnegativity for all decision variables

All money must be invested at all times.

$fund_j - \sum_{i \in sec} x_{i,j} = 0$ for $j = 1,\dots, 7$.

Calculate the amount available to invest in year 1.

$fund_1 = initfund$.

Calculate the amount available to invest in year 2.

$fund_2 = fund_1 - \sum_{i \in sec} x_{i,1} + x_{0,1} * (1+(a_0/100))$.

Calculate the amount available to invest in year 3. 

$fund_3 = fund_2 - \sum_{i \in sec} x_{i,2} + x_{0,2} * (1+(a_0/100)) + x_{1,1} * (1+(a_1/100))$.

Calculate the amount available to invest in year j.

$fund_j = fund_{j-1} - \sum_{i \in sec} x_{i,(j-1)} + x_{0,(j-1)} * (1+(a_0/100)) + x_{1,(j-2)} * (1+(a_1/100)) + x_{2,(j-3)} * (1+(a_2/100))$ for $j = 4,\dots, 8$.





In [1]:
from gurobi import *
initfund = 1000
T=7
a = [.01,.025,.04]
# the total yield at maturity for security 0,1,2, expressed as a percentage init = 1000
sec = range(len(a))
time = range(1,T+2)


#create decision variables
m = Model()
x = {}
fund = {}
for j in time:
    for i in sec:
        x[(i,j)] = m.addVar(lb= 0 , name = 'x_{},{}'.format(i,j), obj = 1 + (a[i]/100))
    fund[j] = m.addVar(lb = 0, name = 'fund_%d' % (j))  
m.update()
#initialize variables before investment takes place for constraint

m.update()
#create an objective function to maximize return
m.setObjective(fund[T+1])
m.modelSense = GRB.MAXIMIZE
m.update()

m.addConstr(initfund == fund[1])

#all money must be inveseted at all times
for j in range(1,T+1):
    m.addConstr(fund[j] - quicksum(x[(i,j)] for i in sec) == 0)

#calculate fund[2]
m.addConstr(fund[2] == fund[1] - quicksum(x[(i,1)] for i in sec) + x[(0,1)] * (1+ a[0]))

#calculate fund[3]  
m.addConstr(fund[3] == fund[2] - quicksum(x[(i,2)] for i in sec) + 
            x[(0,2)] * (1+ a[0]) + x[(1,1)] * (1+ a[1]))

#caluculate fund[t] t =4,.., 8
for j in range(4,T+2):
    m.addConstr(fund[j] == fund[j-1] - quicksum(x[(i,(j-1))] for i in sec) +
               x[(2,(j-3))] * (1+ a[2]) + x[(0,(j-1))] * (1+ a[0]) + x[(1,(j-2))] * (1+ a[1]))
           
m.update()

m.optimize()
print(' ')
print("The maximum yield is %g" % (m.ObjVal) )
print(' ')
for j in time:
    print("The portfolio contained $%g at the beginning of year %d." % (fund[j].X, j))
    for i in sec:
        print("The amount invested in security %d in the beginning of year %d is %g" % (i,j,x[(i,j)].X))
    print(' ')

Optimize a model with 15 rows, 32 columns and 75 nonzeros
Coefficient statistics:
  Matrix range    [1e-02, 1e+00]
  Objective range [1e+00, 1e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+03, 1e+03]
Presolve removed 9 rows and 17 columns
Presolve time: 0.01s
Presolved: 6 rows, 15 columns, 33 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1154000e+30   3.898687e+30   3.115400e+00      0s
       5    1.0926500e+03   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds
Optimal objective  1.092650000e+03
 
The maximum yield is 1092.65
 
The portfolio contained $1000 at the beginning of year 1.
The amount invested in security 0 in the beginning of year 1 is 0
The amount invested in security 1 in the beginning of year 1 is 0
The amount invested in security 2 in the beginning of year 1 is 1000
 
The portfolio contained $0 at the beginning of year 2.
The amount invested in security 0 in the beginning of year 2 is 0